In [1]:
import os, sys
import numpy as np
import torch
import json

sys.path.append("..")
from singleVis.SingleVisualizationModel import SingleVisualizationModel
from singleVis.data import DenseActiveLearningDataProvider
from singleVis.projector import DenseALProjector

/home/xianglin/miniconda3/envs/timevis/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CONTENT_PATH = "/home/xianglin/DVI_data/active_learning/random/resnet18/CIFAR10"
GPU_ID = "1"
epoch_num = 200
iteration = 1

In [3]:
sys.path.append(CONTENT_PATH)
from config import config

In [4]:
SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
BASE_ITERATION =config["BASE_ITERATION"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
LAMBDA = VISUALIZATION_PARAMETER["LAMBDA"]
S_LAMBDA = VISUALIZATION_PARAMETER["S_LAMBDA"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
INIT_NUM = VISUALIZATION_PARAMETER["INIT_NUM"]
ALPHA = VISUALIZATION_PARAMETER["ALPHA"]
BETA = VISUALIZATION_PARAMETER["BETA"]
MAX_HAUSDORFF = VISUALIZATION_PARAMETER["MAX_HAUSDORFF"]
HIDDEN_LAYER = VISUALIZATION_PARAMETER["HIDDEN_LAYER"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
T_N_EPOCHS = VISUALIZATION_PARAMETER["T_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]

# define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")
model = SingleVisualizationModel(input_dims=512, output_dims=2, units=256, hidden_layer=HIDDEN_LAYER)

import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))
data_provider = DenseActiveLearningDataProvider(CONTENT_PATH, net, BASE_ITERATION, epoch_num, split=-1, device=DEVICE, classes=CLASSES,verbose=1)
projector = DenseALProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name="al_hybrid", device=DEVICE)


Finish initialization...


In [ ]:
from singleVis.visualizer import DenseALvisualizer
vis = DenseALvisualizer(data_provider, projector, 300)

In [ ]:
epoch = 200
data = data_provider.train_representation_lb(iteration, epoch)
labels = data_provider.train_labels_lb(iteration)
pred = data_provider.get_pred(iteration, epoch, data).argmax(1)
vis.savefig_cus(iteration, epoch, data, pred, labels)
# vis.savefig(1,200)

In [5]:
samples = np.zeros((200, 50000, 512))
for i in range(1, 201, 1):
    samples[i-1] = data_provider.train_representation(iteration, i)

In [5]:
samples = data_provider.train_representation(iteration, epoch_num)

In [6]:
embeddings_2d = np.zeros((200, 50000, 2))
for e in range(1, 201, 1):
    embeddings_2d[e-1] = projector.batch_project(iteration, e, samples[e-1])
# embeddings_2d = np.transpose(embeddings_2d, [1,0,2])
labels = data_provider.train_labels(iteration)

Successfully load the visualization model in iteration 1 for range (1,2]...
Same range as current visualization model...
Successfully load the visualization model in iteration 1 for range (2,21]...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization m

In [6]:
path = os.path.join(CONTENT_PATH, "Model", "Iteration_{}".format(iteration),"embeddings.npy")
embeddings_2d = np.load(path)

In [7]:
embeddings_2d = np.transpose(embeddings_2d,[1,0,2])

In [8]:
samples.shape,embeddings_2d.shape

((50000, 512), (50000, 200, 2))

In [ ]:
path = os.path.join(CONTENT_PATH, "Model", "Iteration_{}".format(iteration),"embeddings.npy")
np.save(path,embeddings_2d)

In [9]:
from trajectory_manager import TrajectoryManager, FeedbackTrajectoryManager
tm = FeedbackTrajectoryManager(samples, embeddings_2d,30, period=100,metric="v")

In [19]:
tm.clustered()

In [15]:
data = data_provider.train_representation(iteration, epoch_num)
labels = data_provider.train_labels(iteration)
pred = data_provider.get_pred(iteration, epoch_num, data).argmax(1)
wrong_pred_idx = np.argwhere(pred!=labels).squeeze()
wrong_pred_idx.shape

100%|██████████| 250/250 [00:00<00:00, 9119.88it/s]


(18975,)

In [16]:
lb_idx = data_provider.get_labeled_idx(iteration)
ulb_idxs = data_provider.get_unlabeled_idx(50000,lb_idx)

In [20]:
selected = tm.sample_batch(100)
print(len(np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)), len(np.intersect1d(selected, ulb_idxs)))
correct = np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)

52 100


In [21]:
# just sampling
for _ in range(10):
    selected = tm.sample_batch(100, correct)
    print(len(np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)), len(np.intersect1d(selected, ulb_idxs)))
    correct = np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)

46 98
52 99
53 100
51 99
51 99
52 100
54 100
59 100
53 99
50 99


In [16]:
# sim score
for _ in range(10):
    tm.update_belief(correct)
    selected = tm.sample_batch(100, correct)
    print(len(np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)), len(np.intersect1d(selected, ulb_idxs)))
    correct = np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


42 97
35 94
36 97
38 94
42 96
29 95
42 99
33 95
38 98
36 96


In [15]:
# success rate
for _ in range(10):
    tm.update_belief(correct)
    selected = tm.sample_batch(100, correct)
    print(len(np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)), len(np.intersect1d(selected, ulb_idxs)))
    correct = np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


53 99
52 99
56 100
56 99
56 100
45 99
51 100
50 100
55 100
58 99


In [18]:
# hybrid
for _ in range(10):
    tm.update_belief(correct)
    selected = tm.sample_batch(100, correct)
    print(len(np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)), len(np.intersect1d(selected, ulb_idxs)))
    correct = np.intersect1d(np.intersect1d(selected, ulb_idxs), wrong_pred_idx)

59 99
60 99
46 99
49 98
54 99
52 99
54 99
51 99
52 100
59 98


In [40]:
s = np.random.choice(ulb_idxs,size=200,replace=False)

In [41]:
# random
s = np.random.choice(ulb_idxs,size=200,replace=False)
len(np.intersect1d(s, wrong_pred_idx)), len(s)

(79, 200)